# 교차 검증

In [1]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']]
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data,target, random_state=42)

In [2]:
# 의사결정 모델 생성
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)

In [3]:
# 교차 검증 함수
# cross_validate  매개변수 >> 모델객체, 훈련세트, 타깃세트
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input, train_target)
scores
# fit_time, score_time >> 훈련하는 시간, 검증하는 시간
# test_score >> 검증 폴드의 점수

{'fit_time': array([0.00970507, 0.00918198, 0.00793576, 0.00777698, 0.00822973]),
 'score_time': array([0.00240016, 0.00238538, 0.00239491, 0.00226665, 0.00230098]),
 'test_score': array([0.85128205, 0.84820513, 0.8788501 , 0.85112936, 0.84394251])}

In [4]:
# 앞서 train_test_split() 함수로 훈련세트를 섞어서 그냥 진행했음
# 교차 검증 시 훈련 세트를 섞으려면 splitter를 사용해야함 >> KFold splitter 사용
import numpy as np
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
np.mean(scores['test_score'])

0.8546818301479492

In [5]:
# 10-폴드 교차 검증
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
np.mean(scores['test_score'])

0.8585800484734237

교차 검증을 이해했다.

이어서 결정 트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아보자.

이때 테스트 세트를 사용하지 않고 교차 검증을 통해 좋은 모델을 고르면 된다.<br>
(테스트 세트는 제일 마지막에 한 번만 사용!)

# 하이퍼파라미터 튜닝
모델이 스스로 학습하는 파라미터는 모델 파라미터.<br>
사용자가 직접 지정해줘야하는 파라미터는 __하이퍼파라미터__<br>
모델마다 적게는 1~ 2개에서 많게는 5~ 6개의 매개변수를 제공.

여러개의 매개변수값을 여러번 변경해가며 최적의 파라미터를 도출.<br>
사이킷런에서는 이를 도와주는 GridSearchCV 클래스가 있다.

## GridSearchCV
이 클래스는 하이퍼파라미터 탐색과 교차 검증까지 해주기에 cross_validate() 함수가 필요 없다.


In [13]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# min_impurity_decrease의 5개 값마다 5폴드 교차검증 시행 (총 25개의 모델을 훈련)
# n_jobs는 병령 실행에 사용할 CPU 코어 수 지정. -1은 시스템 모든 코어 사용을 의미
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

# GridSearchCV는 25개의 모델 중 검증 점수가 가장 높은 최적의 하이퍼파라미터 모델을 따로 훈련시켜서 따로 저장
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(gs.best_params_)

print(gs.cv_results_['mean_test_score'])

0.9137931034482759
{'min_impurity_decrease': 0.0003}
[0.86843111 0.86925267 0.87315179 0.87212531 0.87130627]


In [16]:
# 조금 더 복잡한 매개변수 조합
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split': range(2,100,10)}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [17]:
# 최적의 하이퍼파라미터
print(gs.best_params_)

# 최상의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 15, 'min_impurity_decrease': 0.0001, 'min_samples_split': 22}
0.8752056020639183


### 랜덤 서치
매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려움.<br>
너무 많은 매개변수 조건이 있어 시간이 오래걸릴 부담 또한 있음.<br>
이때 사용하는 것이 __랜덤 서치__.

랜덤서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 __확률 분포 객체__를 전달.




In [18]:
from scipy.stats import uniform, randint

params = {
          'min_impurity_decrease' : uniform(0.0001,0.001),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1,25)
          }

from sklearn.model_selection import RandomizedSearchCV
# iter=100 >> 위 매개변수 범위에서 총 100번 샘플링하여 교차검증
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f52c5d17510>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f52c30ba450>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f52c33dd490>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f52c5d17d10>},
                   random_state=42)

In [23]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 42, 'min_impurity_decrease': 0.00037864646423661145, 'min_samples_leaf': 1, 'min_samples_split': 2}
0.874178065603117
0.8633846153846154
